In [30]:
import string
import heapq
import collections
import random

In [31]:
words = frozenset(w.strip() for w in open('08-rooms.txt').readlines())
len(words)

2336

In [32]:
def adjacents_explicit(word):
    neighbours = []
    for i in range(len(word)):
        for l in string.ascii_lowercase:
            if l != word[i]:
                neighbours.append(word[0:i] + l + word[i+1:])
    return neighbours

In [33]:
def adjacents(word):
    return frozenset(word[0:i] + l + word[i+1:]
           for i in range(len(word))
           for l in string.ascii_lowercase
           if l != word[i])

In [34]:
neighbours = {w: frozenset(n for n in adjacents(w) if n in words)
             for w in words}

In [35]:
neighbours['abbe']

frozenset({'able'})

In [36]:
neighbours['able']

frozenset({'abbe', 'ably', 'axle'})

In [37]:
def distance(w1, w2):
    return sum(1 for i in range(len(w1))
               if w1[i] != w2[i])

In [38]:
distance('abbe', 'abbe')

0

In [39]:
def extend(chain, closed=None):
    if closed:
        nbrs = set(neighbours[chain[-1]]) - closed
    else:
        nbrs = neighbours[chain[-1]]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [40]:
extend(['abbe'])

[['abbe', 'able']]

In [41]:
extend(['abbe', 'able'])

[['abbe', 'able', 'ably'], ['abbe', 'able', 'axle']]

In [42]:
extend(['abbe', 'able', 'ably'])

[['abbe', 'able', 'ably', 'ally']]

In [119]:
def bfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current, agenda)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None        

In [44]:
def bfs_search_closed(start, goal, debug=False):
    agenda = [[start]]
    closed = set()
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None        

In [45]:
bfs_search('abbe', 'ally', debug=True)

['abbe']
['abbe', 'able']
['abbe', 'able', 'ably']
['abbe', 'able', 'axle']
['abbe', 'able', 'ably', 'ally']


['abbe', 'able', 'ably', 'ally']

In [121]:
def dfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(agenda)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = successors + agenda[1:]
    if finished:
        return current
    else:
        return None        

In [47]:
dfs_search('abbe', 'ally', debug=True)

['abbe']
['abbe', 'able']
['abbe', 'able', 'ably']
['abbe', 'able', 'ably', 'ally']


['abbe', 'able', 'ably', 'ally']

In [48]:
bfs_search('cart', 'vans', debug=True)

['cart']
['cart', 'cast']
['cart', 'hart']
['cart', 'cars']
['cart', 'mart']
['cart', 'wart']
['cart', 'card']
['cart', 'curt']
['cart', 'tart']
['cart', 'care']
['cart', 'cant']
['cart', 'dart']
['cart', 'part']
['cart', 'carp']
['cart', 'cast', 'last']
['cart', 'cast', 'bast']
['cart', 'cast', 'cant']
['cart', 'cast', 'past']
['cart', 'cast', 'mast']
['cart', 'cast', 'fast']
['cart', 'cast', 'case']
['cart', 'cast', 'east']
['cart', 'cast', 'cyst']
['cart', 'cast', 'cash']
['cart', 'cast', 'cask']
['cart', 'cast', 'vast']
['cart', 'cast', 'cost']
['cart', 'hart', 'hard']
['cart', 'hart', 'harm']
['cart', 'hart', 'hare']
['cart', 'hart', 'hurt']
['cart', 'hart', 'haft']
['cart', 'hart', 'halt']
['cart', 'hart', 'mart']
['cart', 'hart', 'wart']
['cart', 'hart', 'hark']
['cart', 'hart', 'tart']
['cart', 'hart', 'harp']
['cart', 'hart', 'dart']
['cart', 'hart', 'part']
['cart', 'cars', 'bars']
['cart', 'cars', 'ears']
['cart', 'cars', 'caps']
['cart', 'cars', 'wars']
['cart', 'cars', 'ca

['cart', 'cars', 'cans', 'vans']

In [49]:
bfs_search('cart', 'vane')

['cart', 'care', 'cane', 'vane']

In [51]:
len(dfs_search('cart', 'vane'))

1494

In [52]:
def astar_search(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if finished:
        return current
    else:
        return None        

In [53]:
astar_search('cart', 'vane', debug=True)

['cart']
['cart', 'cant']
['cart', 'cant', 'cane']
['cart', 'cant', 'cane', 'vane']


['cart', 'cant', 'cane', 'vane']

In [54]:
def astar_search_closed(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    closed = set()
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if finished:
        return current
    else:
        return None        

In [55]:
astar_search_closed('cart', 'vane', debug=True)

['cart']
['cart', 'cant']
['cart', 'cant', 'cane']
['cart', 'cant', 'cane', 'vane']


['cart', 'cant', 'cane', 'vane']

# Mutually-reachable sets

Find the transitive closure of the `neighbours` relation, so we can see which words can be transformed into which other words.

In [69]:
%%timeit
candidates = [set([k] + list(neighbours[k])) for k in neighbours]
reachables = []
while candidates:
    current = set(candidates.pop())
    altered = False
    for other in candidates:
        if current.intersection(other):
            altered = True
            current.update(other)
            candidates.remove(other)
    if altered:
        candidates.append(current)
    else:
        reachables.append(current)

len(reachables)

10 loops, best of 3: 23.9 ms per loop


In [71]:
%%timeit
candidates = set(frozenset({k}.union(neighbours[k])) for k in neighbours)
reachables = set()
while candidates:
    current = set(candidates.pop())
    remove_because_merged = set()
    for other in candidates:
        if current.intersection(other):
            current.update(other)
            remove_because_merged.add(other)
    if remove_because_merged:
        for rbm in remove_because_merged:
            candidates.remove(rbm)
        candidates.add(frozenset(current))
    else:
        reachables.add(frozenset(current))

len(reachables)

10 loops, best of 3: 25.9 ms per loop


In [72]:
len(max(reachables, key=len))

2204

In [73]:
len(min(reachables, key=len))

1

In [74]:
collections.Counter(len(r) for r in reachables)

Counter({1: 75, 2: 6, 3: 7, 4: 2, 5: 2, 6: 1, 2204: 1})

In [75]:
[len(r) for r in reachables if 'abbe' in r]

[5]

In [76]:
[r for r in reachables if 'abbe' in r]

[frozenset({'abbe', 'able', 'ably', 'ally', 'axle'})]

In [77]:
sorted((r for r in reachables if len(r) > 1 if len(r) < 10), key=len)

[frozenset({'bevy', 'levy'}),
 frozenset({'ogle', 'ogre'}),
 frozenset({'demo', 'memo'}),
 frozenset({'crud', 'crux'}),
 frozenset({'thou', 'thru'}),
 frozenset({'idol', 'idyl'}),
 frozenset({'icon', 'ikon', 'iron'}),
 frozenset({'used', 'user', 'uses'}),
 frozenset({'opal', 'oral', 'oval'}),
 frozenset({'also', 'alto', 'auto'}),
 frozenset({'idle', 'idly', 'isle'}),
 frozenset({'afar', 'agar', 'ajar'}),
 frozenset({'eddy', 'edge', 'edgy'}),
 frozenset({'each', 'etch', 'inch', 'itch'}),
 frozenset({'high', 'nigh', 'sigh', 'sign'}),
 frozenset({'info', 'into', 'onto', 'undo', 'unto'}),
 frozenset({'abbe', 'able', 'ably', 'ally', 'axle'}),
 frozenset({'ache', 'achy', 'acme', 'acne', 'acre', 'ashy'})]

In [78]:
'; '.join(sorted(list(r)[0] for r in reachables if len(r) == 1))

'adze; agog; ague; ahoy; alga; ammo; amok; anal; ankh; apse; aqua; aura; avow; awol; bozo; ebbs; echo; ecru; emus; ends; envy; epee; epic; espy; euro; evil; exam; expo; guru; hymn; ibex; iffy; imam; iota; isms; judo; kiwi; liar; luau; lynx; mayo; meow; myna; nova; obey; oboe; odor; ohms; okra; oleo; once; onyx; orgy; ovum; rely; rhea; semi; sexy; stye; sync; taxi; tofu; tuft; tutu; twos; ugly; ulna; upon; urge; uric; urns; void; wiki; yeti; zebu'

In [120]:
bfs_search('ache', 'ashy', debug=True)

['ache'] [['ache']]
['ache', 'acne'] [['ache', 'acne'], ['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy']]
['ache', 'acme'] [['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy'], ['ache', 'acne', 'acme'], ['ache', 'acne', 'acre']]
['ache', 'acre'] [['ache', 'acre'], ['ache', 'achy'], ['ache', 'acne', 'acme'], ['ache', 'acne', 'acre'], ['ache', 'acme', 'acne'], ['ache', 'acme', 'acre']]
['ache', 'achy'] [['ache', 'achy'], ['ache', 'acne', 'acme'], ['ache', 'acne', 'acre'], ['ache', 'acme', 'acne'], ['ache', 'acme', 'acre'], ['ache', 'acre', 'acne'], ['ache', 'acre', 'acme']]
['ache', 'acne', 'acme'] [['ache', 'acne', 'acme'], ['ache', 'acne', 'acre'], ['ache', 'acme', 'acne'], ['ache', 'acme', 'acre'], ['ache', 'acre', 'acne'], ['ache', 'acre', 'acme'], ['ache', 'achy', 'ashy']]
['ache', 'acne', 'acre'] [['ache', 'acne', 'acre'], ['ache', 'acme', 'acne'], ['ache', 'acme', 'acre'], ['ache', 'acre', 'acne'], ['ache', 'acre', 'acme'], ['ache', 'achy', 'ashy'], ['ache', 'acne', 'acme', '

['ache', 'achy', 'ashy']

In [122]:
dfs_search('ache', 'ashy', debug=True)

[['ache']]
[['ache', 'acne'], ['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acne', 'acme'], ['ache', 'acne', 'acre'], ['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acne', 'acme', 'acre'], ['ache', 'acne', 'acre'], ['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acne', 'acre'], ['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acne', 'acre', 'acme'], ['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acme'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acme', 'acne'], ['ache', 'acme', 'acre'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acme', 'acne', 'acre'], ['ache', 'acme', 'acre'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acme', 'acre'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acme', 'acre', 'acne'], ['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acre'], ['ache', 'achy']]
[['ache', 'acre', 'acne'], ['ache', 'acre', 'acme'], ['ache', 'achy']]
[['ache', 'acre', 'acne', 'acme'], ['ache

['ache', 'achy', 'ashy']

In [79]:
astar_search('buns', 'punk')

['buns', 'bunk', 'punk']

In [80]:
for a in reachables:
    for b in reachables:
        if a != b:
            if not a.isdisjoint(b):
                print(a, b)

In [81]:
# longest_chain = []
# with open('all-chains-4.txt', 'w', 1) as f:
#     for ws in reachables:
#         for s in ws:
#             for t in ws:
#                 if s < t:
#                     chain = astar_search(s, t)
#                     if chain:
#                         f.write('{}\n'.format(chain))
#                         if len(chain) > len(longest_chain):
#                             longest_chain = chain

# longest_chain

In [82]:
bigset = max(reachables, key=len)

In [83]:
for _ in range(20):
    start, goal = random.sample(bigset, 2)
    print(astar_search_closed(start, goal))

['foil', 'boil', 'bail', 'ball', 'balk', 'bank', 'hank']
['pint', 'pent', 'peat', 'meat', 'meal', 'mewl']
['fuse', 'fuss', 'furs', 'ours']
['plop', 'flop', 'flow', 'flew']
['lull', 'bull', 'bell', 'belt', 'welt', 'wept', 'kept']
['coma', 'coda', 'cods', 'cons', 'sons']
['pits', 'bits', 'bats', 'bass', 'bash', 'rash']
['bean', 'been', 'bees', 'beds', 'buds', 'suds']
['yips', 'nips', 'nits', 'nite', 'note', 'vote']
['pent', 'pens', 'yens']
['crop', 'coop', 'coos', 'cons', 'cans', 'fans']
['flop', 'clop', 'coop', 'cool', 'cowl', 'bowl', 'bawl']
['hill', 'hull', 'hulk', 'sulk', 'suck']
['aced', 'acid', 'arid', 'grid', 'grin', 'gain', 'rain']
['land', 'sand', 'sans', 'suns', 'subs', 'hubs']
['cogs', 'coos', 'cool', 'coal', 'foal', 'foam']
['loss', 'moss', 'mods', 'nods', 'nous', 'yous']
['soap', 'soar', 'boar', 'boas', 'bogs', 'jogs']
['musk', 'must', 'bust', 'best', 'beet', 'beef']
['fine', 'wine', 'wane', 'want', 'waft']


In [84]:
astar_search('cops', 'thug')

['cops', 'coos', 'coon', 'coin', 'chin', 'thin', 'this', 'thus', 'thug']

In [85]:
[len(r) for r in reachables if 'love' in r if 'hate' in r]

[2204]

In [86]:
[len(r) for r in reachables if 'stay' in r ]

[2204]

In [87]:
astar_search('hate', 'love')

['hate', 'have', 'hove', 'love']

In [88]:
astar_search('wars', 'love')

['wars', 'ware', 'wave', 'wove', 'love']

In [89]:
%time len(astar_search('wars', 'love'))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 199 µs


5

In [90]:
%time len(astar_search_closed('wars', 'love'))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 210 µs


5

In [91]:
%time len(dfs_search('wars', 'love'))

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 13.5 ms


272

In [92]:
# %time len(bfs_search('wars', 'love'))

In [93]:
%time len(bfs_search_closed('wars', 'love'))

CPU times: user 724 ms, sys: 0 ns, total: 724 ms
Wall time: 723 ms


5

In [94]:
astar_search('fear', 'love')

['fear', 'feat', 'fest', 'lest', 'lost', 'lose', 'love']

In [95]:
astar_search('fail', 'pass')

['fail', 'fall', 'pall', 'pals', 'pass']

In [96]:
astar_search('star', 'born')

['star', 'soar', 'boar', 'boor', 'boon', 'born']

In [97]:
astar_search('open', 'pass')

['open',
 'oven',
 'even',
 'eves',
 'eyes',
 'byes',
 'bees',
 'begs',
 'bags',
 'bass',
 'pass']

In [98]:
neighbours['pass']

frozenset({'bass',
           'lass',
           'mass',
           'pads',
           'pals',
           'pans',
           'paps',
           'pars',
           'past',
           'pats',
           'paws',
           'pays',
           'puss',
           'sass'})

In [99]:
[len(r) for r in reachables if 'exam' in r]

[1]

In [100]:
[len(r) for r in reachables if 'star' in r if 'born' in r]

[2204]

In [101]:
%%timeit
astar_search('bats', 'exit')

1 loop, best of 3: 9.05 s per loop


In [102]:
%%timeit
astar_search_closed('bats', 'exit')

10 loops, best of 3: 147 ms per loop


In [103]:
astar_search_closed('bats', 'exit')

['bats',
 'bans',
 'band',
 'sand',
 'said',
 'skid',
 'skit',
 'smit',
 'emit',
 'exit']

In [104]:
# solutions = {}
# for _ in range(10000):
#     start, goal = random.sample(bigset, 2)
#     solution = astar_search_closed(start, goal)
#     sl = len(solution)
#     if sl not in solutions:
#         solutions[sl] = []
#     if len(solutions[sl]) < 4:
#         solutions[sl].append([start, goal])
        
# #     if len(solution) >= 10:
# #         solutions += [solution]
        
# solutions

In [105]:
solutions = {2: [['heel', 'keel'], ['wane', 'wave'], ['cell', 'sell'], ['cons', 'cobs']],
 3: [['hank', 'haws'], ['bars', 'bets'], ['rats', 'paws'], ['lock', 'hack']],
 4: [['rule', 'sore'], ['wavy', 'rape'], ['peas', 'ping'], ['bond', 'toll']],
 5: [['cope', 'yowl'], ['lose', 'loci'], ['rump', 'dash'], ['four', 'dyes']],
 6: [['boon', 'sell'], ['lots', 'pomp'], ['cola', 'turn'], ['boos', 'laid']],
 7: [['eave', 'inns'], ['meek', 'mere'], ['keys', 'wily'], ['slam', 'yore']],
 8: [['hack', 'flip'], ['crag', 'huge'], ['flux', 'gill'], ['play', 'busy']],
 9: [['lacy', 'whey'], ['wren', 'rook'], ['lire', 'drip'], ['grab', 'lame']],
 10: [['over', 'turn'], ['worn', 'anew'], ['stow', 'elks'], ['ergo', 'rich']],
 11: [['bask', 'idea'], ['gabs', 'thud'], ['idea', 'clod'], ['mark', 'ibis']],
 12: [['umps', 'torn'], ['futz', 'shun'], ['abut', 'face'], ['slug', 'open']],
 13: [['umps', 'skin'], ['chum', 'rats'], ['fury', 'chum'], ['omen', 'zany']],
 14: [['chug', 'gaff'], ['atom', 'fizz'], ['chug', 'jinn'], ['amen', 'flog'],
     ['buzz', 'grog'], ['imps', 'pros']],
 15: [['chug', 'oxen'], ['amen', 'doff']]}

In [106]:
%%timeit
astar_search_closed('blab', 'amen')

1 loop, best of 3: 358 ms per loop


In [107]:
%time len(astar_search_closed('blab', 'amen'))

CPU times: user 384 ms, sys: 0 ns, total: 384 ms
Wall time: 382 ms


14

In [108]:
%time len(astar_search_closed('amen', 'doff'))

CPU times: user 112 ms, sys: 0 ns, total: 112 ms
Wall time: 109 ms


15

In [109]:
%time len(astar_search_closed('chug', 'jinn'))

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 36.5 ms


14

In [110]:
%time len(astar_search_closed('amen', 'flog'))

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 18.1 ms


14

In [111]:
%time len(astar_search_closed('buzz', 'grog'))

CPU times: user 272 ms, sys: 0 ns, total: 272 ms
Wall time: 267 ms


14

In [112]:
%time len(astar_search_closed('imps', 'pros'))

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 37.8 ms


14

In [113]:
nb_count = collections.Counter()
for w in neighbours['rash']:
    for w2 in neighbours[w]:
        if w2 == 'bush': print(w, w2, w2 not in neighbours['rash'])
        if w2 != 'rash' and w2 not in neighbours['rash']:
            nb_count.update([w2])
nb_count.most_common(1)[0][1]

rush bush True
bash bush True


2

In [114]:
[w for w in neighbours['bush'] if w in nb_count]

['gush', 'mush', 'bosh', 'tush', 'lush', 'push', 'hush']

In [115]:
best_start = ''
best_overlap_count = 0
for w0 in neighbours:    
    nb_count = collections.Counter()
    for w in neighbours[w0]:
        for w2 in neighbours[w]:
            if w2 != w0 and w2 not in neighbours[w0]:
                nb_count.update([w2])
    if len(nb_count) > 0:
        if nb_count.most_common(1)[0][1] > best_overlap_count:
            best_start = w0
            best_overlap_count = nb_count.most_common(1)[0][1]
        
best_start, best_overlap_count

('came', 2)

In [116]:
w0

'hags'

In [117]:
set(neighbours['rash']) & set(neighbours['bush'])

{'bash', 'rush'}